In [39]:
import numpy as np
import scipy as scipy
from numpy.random import uniform
import scipy.stats

def create_uniform_particles(x_range, y_range, N):
    particles = np.empty((N, 2))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N)
    return particles

def predict(particles, u, std, dt=1.):
    N = len(particles)
    dist = (u[0] * dt) + (np.random.randn(N) * std[0])
    particles[:, 0] +=  dist
    particles[:, 1] +=  dist
    
def update(particles, weights, z, R, landmarks):
    weights.fill(1.)
    for i, landmark in enumerate(landmarks):
       
        distance=np.power((particles[:,0] - landmark[0])**2 +(particles[:,1] - landmark[1])**2,0.5)
        weights *= scipy.stats.norm(distance, R).pdf(z[i])
 
 
    weights += 1.e-300 # avoid round-off to zero
    weights /= sum(weights)
    
def neff(weights):
    return 1. / np.sum(np.square(weights))
 
def systematic_resample(weights):
    N = len(weights)
    positions = (np.arange(N) + np.random.random()) / N
 
    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N and j<N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return indexes
    
def estimate(particles, weights):
    pos = particles[:, 0:1]
    mean = np.average(pos, weights=weights, axis=0)
    var = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var
 
def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights /= np.sum(weights)
    
center=np.array([[-10,-10]]) 
trajectory=np.zeros(shape=(0,2))
x_range=np.array([0,15])
y_range=np.array([0,15])
x = 7.5
y = 4.8
#Number of partciles
N=10000
H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
landmarks = H
NL = len(landmarks)
particles=create_uniform_particles(x_range, y_range, N)
weights = np.array([1.0]*N)
previous_x=1
previous_y=1
sensor_std_err=1
k = 1

while k<200:
    global center
    global trajectory
    global previous_x
    global previous_y
    global zs

    center=np.array([[x,y]])
    trajectory=np.vstack((trajectory,np.array([x,y])))
    #noise=sensorSigma * np.random.randn(1,2) + sensorMu

    if previous_x >0:
        distance=np.linalg.norm(np.array([[previous_x,previous_y]])-np.array([[x,y]]) ,axis=1)
        std=np.array([1,1])
        u=np.array([distance])
        predict(particles, u, std, dt=1.)
        zs = (np.linalg.norm(landmarks - center, axis=1) + (np.random.randn(NL) * sensor_std_err))
        update(particles, weights, z=zs, R=1, landmarks=landmarks)

        indexes = systematic_resample(weights)
        resample_from_index(particles, weights, indexes)
        k = k+1

    previous_x=x
    previous_y=y
    
print(particles[0,0])
print(particles[0,1])


7.265371836807271
4.428689481915411
